<a href="https://colab.research.google.com/github/TusharJiShukla/CS-307-Lab-Report/blob/main/Lab_08_In_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import copy

class GridWorldMDP:
    def __init__(self, rows=3, cols=4, wall_coords=(1, 1), terminals=None):
        self.rows = rows
        self.cols = cols
        self.wall = wall_coords
        self.terminals = terminals if terminals else [(1, 3), (2, 3)]
        self.actions = ['U', 'D', 'L', 'R']
        # Configuration for stochastic movement: (Intended, Left-deviation, Right-deviation)
        self.noise_probs = {'intended': 0.8, 'deviation': 0.1}

    def _is_accessible(self, r, c):
        """Checks if a coordinate is within bounds and not a wall."""
        if (r, c) == self.wall:
            return False
        return 0 <= r < self.rows and 0 <= c < self.cols

    def _get_next_state(self, curr_r, curr_c, move):
        """Determines the next state given a deterministic move."""
        next_r, next_c = curr_r, curr_c

        if move == 'U': next_r += 1
        elif move == 'D': next_r -= 1
        elif move == 'L': next_c -= 1
        elif move == 'R': next_c += 1

        if self._is_accessible(next_r, next_c):
            return next_r, next_c
        return curr_r, curr_c

    def _get_stochastic_neighbors(self, action):
        """Returns the intended move and the two perpendicular deviation moves."""
        # Mapping perpendicular directions relative to action
        mapping = {
            'L': {'left': 'D', 'right': 'U'},
            'R': {'left': 'U', 'right': 'D'},
            'U': {'left': 'L', 'right': 'R'},
            'D': {'left': 'R', 'right': 'L'}
        }
        return mapping[action]['left'], mapping[action]['right']

    def calculate_bellman_update(self, r, c, grid_values, grid_rewards, gamma=1.0):
        """Calculates the updated value for a specific state."""
        state_value_accumulator = 0

        # Iterating through all 4 possible actions (Uniform Policy: 0.25 probability)
        for action in self.actions:
            # 1. Intended direction
            nxt_r, nxt_c = self._get_next_state(r, c, action)
            val_intended = grid_rewards[nxt_r][nxt_c] + gamma * grid_values[nxt_r][nxt_c]

            # 2. Perpendicular deviations
            left_act, right_act = self._get_stochastic_neighbors(action)

            lr, lc = self._get_next_state(r, c, left_act)
            val_left = grid_rewards[lr][lc] + gamma * grid_values[lr][lc]

            rr, rc = self._get_next_state(r, c, right_act)
            val_right = grid_rewards[rr][rc] + gamma * grid_values[rr][rc]

            # Weighted sum for this specific action based on environment noise
            action_utility = (val_intended * self.noise_probs['intended'] +
                              val_left * self.noise_probs['deviation'] +
                              val_right * self.noise_probs['deviation'])

            state_value_accumulator += action_utility * 0.25

        return state_value_accumulator

    def run_solver(self, step_reward, epsilon=1e-8):
        value_grid = [[0.0 for _ in range(self.cols)] for _ in range(self.rows)]

        reward_grid = [[step_reward for _ in range(self.cols)] for _ in range(self.rows)]
        reward_grid[1][3] = -1  # Terminal state -1
        reward_grid[2][3] = 1   # Terminal state +1

        iterations = 0
        while True:
            max_delta = 0

            for r in range(self.rows):
                for c in range(self.cols):
                    if (r, c) in self.terminals or (r, c) == self.wall:
                        continue

                    old_v = value_grid[r][c]
                    new_v = self.calculate_bellman_update(r, c, value_grid, reward_grid)
                    value_grid[r][c] = new_v

                    max_delta = max(max_delta, abs(old_v - new_v))

            iterations += 1
            if max_delta < epsilon:
                print(f"Converged in {iterations} iterations")
                break

        self.display_grid(value_grid)

    def display_grid(self, grid):
        for r in range(self.rows - 1, -1, -1):
            row_str = "|"
            for c in range(self.cols):
                row_str += f" {grid[r][c]:.2f}\t|"
            print("-" * 40)
            print(row_str)
        print("-" * 40 + "\n")


print("MDP Value Functions (Logic Preserved)\n")
experiment_rewards = [-0.04, -2, 0.1, 0.02, 1]

solver = GridWorldMDP()

for r_val in experiment_rewards:
    print(f"Computing for Step Reward r(s) = {r_val}")
    solver.run_solver(step_reward=r_val)

MDP Value Functions (Logic Preserved)

Computing for Step Reward r(s) = -0.04
Converged in 312 iterations
----------------------------------------
| -1.23	| -0.83	| -0.28	| 0.00	|
----------------------------------------
| -1.47	| 0.00	| -0.87	| 0.00	|
----------------------------------------
| -1.55	| -1.47	| -1.22	| -1.17	|
----------------------------------------

Computing for Step Reward r(s) = -2
Converged in 384 iterations
----------------------------------------
| -59.71	| -46.01	| -24.32	| 0.00	|
----------------------------------------
| -65.41	| 0.00	| -21.94	| 0.00	|
----------------------------------------
| -63.10	| -52.80	| -34.49	| -20.75	|
----------------------------------------

Computing for Step Reward r(s) = 0.1
Converged in 324 iterations
----------------------------------------
| 2.95	| 2.39	| 1.44	| 0.00	|
----------------------------------------
| 3.10	| 0.00	| 0.63	| 0.00	|
----------------------------------------
| 2.85	| 2.20	| 1.15	| 0.23	|
---------------